In [1]:
import os
import re
from pprint import pprint

from lxml import html
from lxml.etree import tostring

import pandas as pd

In [2]:
files = os.listdir(path="data")
files = [f for f in files if '.html' in f]
files

['doc-01.01.2014.html', 'doc-01.01.2015.html', 'doc-01.01.2013.html']

In [3]:
class DFprocessor:
        
    def struct_split(df):
        titles = list(df[df.keys()[0]])

        high_keys = [key for key in titles if not 'I' in key.split('.')[0]]
        low_keys =  [key for key in titles if 'I' in key.split('.')[0]]

        high_indexes = [titles.index(key) for key in high_keys]
        low_indexes = [titles.index(key) for key in low_keys]

        return high_indexes, low_indexes
    
    def json_composer(high_indexes, low_indexes, df):
        titles = list(df[df.keys()[0]])
        
        json = []
        
        for i in range(len(titles) - 1):
            if i in high_indexes:

                outer_json = {
                    "name": titles[i],
                    "digit": '{:.2f}'.format(df[df.keys()[2]][i]),
                    "rows": []
                }
                json.append(outer_json)
        
            elif i in low_indexes:
        
                inner_json = {
                    "name": titles[i],
                    "digit": '{:.2f}'.format(df[df.keys()[1]][i]),
                    "rows": []
                }
                outer_json["rows"].append(inner_json)
                
        else:
            json.append({
                    "name": titles[i + 1],
                    "digit": df[df.keys()[2]][i + 1],
                    "is_summa": True
                })

        return json

In [4]:
structure = {"balances": []}
for filename in files:
    with open('data/' + filename) as file:
        file_data = file.read()
        root_sub = re.sub(r'<\/?.*?>', '', file_data)
        
    root_sub = re.sub(r'\s+', ' ', root_sub)
    root_sub = re.sub(r'&amp;', '&', root_sub)
    root_sub = re.sub(r'&lt;', '<', root_sub)
    root_sub = re.sub(r'&gt;', '>', root_sub)
    root_sub = re.sub(r'\n', '', root_sub)
    root_sub = re.sub(r'&ouml;', 'ö', root_sub)
    root_sub = re.sub(r'&uuml;', 'ü', root_sub)
    root_sub = re.sub(r'&euro;', '€', root_sub)

    nums_old = re.findall(r'[0-9]*\.*[0-9]{1,3}\.[0-9]{3}[,\.][0-9]{2}', root_sub)
    
    nums_new = []
    for n in nums_old:
        nums_new.append(re.sub('[,\.]', '', n)[:-2] + '.' + n[-2:])
    
    nums = zip(nums_old, nums_new)
    
    for item in nums:
        root_sub = re.sub(item[0], item[1], root_sub)
    
    root = html.fromstring(root_sub)
    
    aktiv_table = root.xpath('//table[@class="std_table"]')[0]
    passiv_table = root.xpath('//table[@class="std_table"]')[1]
    aktiv_df = pd.read_html(tostring(aktiv_table))[0]
    passiv_df = pd.read_html(tostring(passiv_table))[0]

    year = int(filename.split('.')[-2])

    processor = DFprocessor

    aktiv_high_indexes, aktiv_low_indexes = processor.struct_split(aktiv_df)
    passiv_high_indexes, passiv_low_indexes = processor.struct_split(passiv_df)

    aktiv_json = processor.json_composer(aktiv_high_indexes, aktiv_low_indexes, aktiv_df)
    passiv_json = processor.json_composer(passiv_high_indexes, passiv_low_indexes, passiv_df)

    json = {
        "year": year,
        "balance": {
            "aktiva": aktiv_json,
            "passiva": passiv_json,
        }
    }

    structure['balances'].append(json)

In [5]:
structure

{'balances': [{'year': 2014,
   'balance': {'aktiva': [{'name': 'A. Anlagevermögen',
      'digit': '345777.00',
      'rows': [{'name': 'I. Sachanlagen', 'digit': '345777.00', 'rows': []}]},
     {'name': 'B. Umlaufvermögen',
      'digit': '662776.67',
      'rows': [{'name': 'I. Forderungen und sonstige Vermögensgegenstände',
        'digit': '573742.76',
        'rows': []},
       {'name': 'II. Kassenbestand, Bundesbankguthaben, Guthaben bei Kreditinstituten und Schecks',
        'digit': '89033.91',
        'rows': []}]},
     {'name': 'C. Rechnungsabgrenzungsposten',
      'digit': '12298.08',
      'rows': []},
     {'name': 'Summe Aktiva', 'digit': 1020851.75, 'is_summa': True}],
    'passiva': [{'name': 'A. Eigenkapital',
      'digit': '614604.72',
      'rows': [{'name': 'I. Gezeichnetes Kapital',
        'digit': '120000.00',
        'rows': []},
       {'name': 'II. Bilanzgewinn', 'digit': '494604.72', 'rows': []}]},
     {'name': 'B. Rückstellungen', 'digit': '284759.12'